In [ ]:
from modules.helpers.runners import run_solver_against_configs
from modules.helpers.graph_builder_helpers import load_graph_from_file
from modules.helpers.graph_builder_helpers import get_stats_for_nodes
from modules.Simulator import *

In [ ]:
import random
import math
import os, json
import pandas as pd

In [ ]:
def save_list_as_csv(path, items):
    with open(path, 'w') as f:
        for item in items:
            f.write("%s " % item)

In [ ]:
GRAPH_PATH = ''
FULL_GRAPH_PATH = ''
SEED_FILE = ''
FULL_SEED_FILE = ''

simmulations_to_run = 10
threads_to_use = 12
desired_size_for_candidates = 15
nodes_to_pick = 10
picks_to_do = 100

VERSUS_METHODS = ['SparseShield','Dom']
VERSUS_OUT_NODES = 'versus_out/'
VERSUS_OUT_RESULTS = 'versus_results/'

In [ ]:
G = load_graph_from_file(FULL_GRAPH_PATH)

In [ ]:
(degrees, max_degree, min_degree, avg_degree) = get_stats_for_nodes(G)

In [ ]:
pick_from = []
done = False

for degree, nodes in sorted(degrees.items(), key=lambda item: item[0], reverse = True):
    if done:
        break

    for node in nodes:
        pick_from.append(node)

        if (len(pick_from) == desired_size_for_candidates):
            done = True
            break

In [ ]:
main_columns = ['id', 'nodes_in_graph', 'edges_in_graph','total_time', 'nodes_to_block', 'simulation_name']
secondary_columns = ['simulation_index', 'node']

In [ ]:
index = 6001
idx = 6001

jsons_data = pd.DataFrame(columns=main_columns)
json_blocked_data = pd.DataFrame(columns=secondary_columns)

for _ in range(picks_to_do):
    seeds = list(random.sample(pick_from, nodes_to_pick))

    save_list_as_csv(FULL_SEED_FILE, seeds)
    run_solver_against_configs(results_path=VERSUS_OUT_NODES, graph_file=GRAPH_PATH, seed_file=SEED_FILE, startNumber=1, endNumber=11, step=1, algorithms_to_run=VERSUS_METHODS, just_solve = True)
    sim = Simulator(G, seeds)
    run_solver_against_configs(results_path=VERSUS_OUT_NODES, graph_file=GRAPH_PATH, seed_file=SEED_FILE, startNumber=10, endNumber=950, step=100, algorithms_to_run=VERSUS_METHODS, just_solve = True)
    sim = Simulator(G, seeds)
    
    json_files = [pos_json for pos_json in os.listdir(VERSUS_OUT_NODES) if pos_json.endswith('.json')]

    for _, file_name in enumerate(json_files):
        tokens = file_name.replace('.json', '').split('_')
        nodes_to_block = int(tokens[1])
        name = tokens[2]
        graph_stats = tokens[3].split(']')
        num_nodes = int(graph_stats[0].replace('[',''))
        num_edges = int(graph_stats[1].replace('[',''))
        full_path = os.path.join(VERSUS_OUT_NODES, file_name)

        with open(full_path) as json_file:
            json_text = json.load(json_file)
            jsons_data.loc[index] = [index, num_nodes, num_edges, json_text['Total time'], nodes_to_block, name]
            blocked_nodes = json_text['Blocked nodes']
            sim.add_blocked(index, blocked_nodes)

            for blocked_node in blocked_nodes:
                json_blocked_data.loc[idx] = [index, blocked_node]
                idx += 1

        index += 1
        os.remove(full_path)

    results = sim.run(simmulations_to_run, threads_to_use)
    file_id = 'data' + str(int(index / 2)) + '.json'
    out_path = os.path.join(VERSUS_OUT_RESULTS, file_id)
    
    with open(out_path, 'w') as f:
        json.dump(results, f)
        

jsons_data.to_csv(os.path.join(VERSUS_OUT_RESULTS, 'simulationData.csv'), index=False)
json_blocked_data.to_csv(os.path.join(VERSUS_OUT_RESULTS, 'simulationNodes.csv'), index=False)

In [ ]:
json_files = [pos_json for pos_json in os.listdir(VERSUS_OUT_RESULTS) if pos_json.endswith('.json')]
sim_data = pd.DataFrame(columns=['simulation_index', 'mean_activated', 'var_activated', 'mean_saved', 'var_saved', 'saved_nodes', 'activated_nodes'])
index = 0

for _, js in enumerate(json_files):
    full_path = os.path.join(VERSUS_OUT_RESULTS, js)

    with open(full_path) as json_file:
        json_text = json.load(json_file)
        for sim_id in json_text['solvers'].keys():
            mean_activated = json_text['solvers'][sim_id]['activated nodes']['mean']
            var_activated = json_text['solvers'][sim_id]['activated nodes']['var']
            mean_saved = json_text['solvers'][sim_id]['saved nodes']['mean']
            var_saved = json_text['solvers'][sim_id]['saved nodes']['var']

            sim_data.loc[index] = [sim_id, mean_activated, var_activated, mean_saved, mean_activated, int(math.ceil(mean_saved)), int(math.ceil(mean_activated))]
            index += 1

    os.remove(full_path)

sim_data.to_csv(os.path.join(VERSUS_OUT_RESULTS, 'simulationStats.csv'), index=False)